In [84]:
from IPython.display import clear_output
# Descarga de csv
!wget https://www.dropbox.com/s/nb2cnlgjx93es92/tweets_2022_abril_junio.csv.zip
!unzip tweets_2022_abril_junio.csv.zip
clear_output()

!pip3 uninstall spacy
!pip3 install spacy

!pip3 uninstall spacymoji
!pip3 install spacymoji

!spacy download es_core_news_lg
!spacymoji download nlp_en
clear_output()

In [85]:
import numpy as np
import pandas as pd
import itertools
import re
import spacy
from spacymoji import Emoji
from tqdm import tqdm


### Objetivo
Encontrar personas que escribar tweets de manera similar

In [86]:
clean_tweets = pd.read_csv('tweets_2022_abril_junio.csv') 
print("Numero total de tweets", clean_tweets.shape)

Numero total de tweets (4594980, 6)


# 1. Prepocesamiento de datos

In [87]:
# Eliminar tweets duplicados de un mismo usuario
clean_tweets = clean_tweets.drop_duplicates(subset=['screen_name', 'text'], keep='last')
# Eliminar ids duplicados
clean_tweets = clean_tweets.drop_duplicates(subset=['id'], keep='last')
# Eliminar RTs
clean_tweets = clean_tweets[clean_tweets['text'].str.startswith("RT ") == False]

clean_tweets = clean_tweets.set_index(['id'])
clean_tweets = clean_tweets[['screen_name', 'text']]
clean_tweets = clean_tweets[clean_tweets['text'].str.startswith("RT ") == False]
print("Numero total de tweets a procesar", clean_tweets.shape)

Numero total de tweets a procesar (1250610, 2)


In [88]:
# Tamaño shingling
k = 10

In [89]:
def format_tweets(text):
    # Tweets en minuscula
    # Eliminar #, @, links, risa
    # Descartas tweets con menos de k caracteres
    text = text.lower()
    text = re.sub(r'#\w+\s?|@\w+\s?|htps:/t.co/\S+|https://t.co/\S+|j\w*j\w*j\w*|a*ja+j[ja]*|lo+l|\n', '', text)
    if len(text) < k + 1:
        return ''
    return text

In [90]:
clean_tweets["preprocess"] = clean_tweets[['text']].apply(lambda x: format_tweets((x.values[0])), axis=1,)
clean_tweets = clean_tweets[clean_tweets['preprocess'] != "" ]

# Eliminar tweets duplicados de un mismo usuario
clean_tweets = clean_tweets.drop_duplicates(subset=['screen_name', 'preprocess'], keep='last')
print("Numero total de tweets a procesar", clean_tweets.shape)

Numero total de tweets a procesar (980092, 3)


# 2. Funciones auxiliares

In [91]:
def shingling(text, k = 3):
    shingles = set()
    total = 0
    for i in range(len(text) - k):
        shingle = text[i:i+k]
        shingles.add(hash(shingle))
        total += 1
    return shingles, total

In [92]:
import random
import collections

def crear_hash(a, b, p, n):
    def f(x):
        return ((a * x + b) % p) % n
    return f

h = []
n = 45991841 # Estimado apriori. Valor mayor al numero total de Shingles
p = 45991889 # Primo mas cercano a n
num_hash = 20 # Numero de funciones de hash
for i in range(num_hash):
    a = random.randint(1, p - 1)
    b = random.randint(1, p - 1)
    h.append(crear_hash(a, b, p, n))

# 3. MinHash

In [94]:
import collections
b = 5  # número de bandas
buckets = collections.defaultdict(list)
# Almacena todas las firmas MinHash
signatures = {} 

# Para cada tweet
for tweet_id in tqdm(clean_tweets.index):
    # Se crean los shingles y se calculan las firmas minhash
    shingleSet, total = shingling(clean_tweets.loc[tweet_id]['preprocess'], k)
    signature = []
    for i in range(num_hash):
        # Se obtiene el minhash para cada shingle
        minhashCode = p + 1
        for shi in shingleSet:
            hashCode = h[i](shi)
            if hashCode < minhashCode:
                minhashCode = hashCode
        # Se almacenan posibles candidatos
        signature.append(minhashCode)
        buckets[minhashCode].append(tweet_id)

    # Se almacenan todas las firmas MinHash
    signatures[tweet_id] = signature 

100%|██████████| 980092/980092 [06:34<00:00, 2487.34it/s]


# 4. Tweets similares

In [96]:
def jaccard(a, b):
    return len(a.intersection(b)) / len(a.union(b))

def get_similar_tweets(id, t):

    # Un tweet dado se compara unicamente con los otros tweets con los que 
    # potencialemente puede ser similar 
    tweet = clean_tweets.loc[id]['preprocess']
    similar = set()
    signature1 = signatures[id]

    for minhashCode in signature1:
        bucket = buckets[minhashCode]
        for other_tweet in bucket:
            signature2 = signatures[other_tweet]

            # No todos los candidatos terminan siendo similares
            # Depende del valor de t
            jac = jaccard(set(signature1), set(signature2))
            if jac > t:
                similar.add(other_tweet)
    return similar

def show_similar_tweets(tweets_id):
    max_len = max(len(clean_tweets.loc[id]['screen_name']) for id in tweets_id)
    for id in tweets_id:
        name = clean_tweets.loc[id]['screen_name']
        text = clean_tweets.loc[id]['text']
        print("@{:<{}} : {}".format(name, max_len, text))

In [110]:
tweet_data = clean_tweets.loc[1528429011369488384]
print("@{:<{}} : {}".format(tweet_data['screen_name'], len(tweet_data['screen_name']), tweet_data['text']))

@dangubbly : @christianpviera Claro, un Chile de ultra izquierda. Despierta Chile !!!! No queremos un país de ultra izquierda para la@ultra izquierda.


In [112]:
t = 0.1
tw = get_similar_tweets(1528429011369488384, t)
show_similar_tweets(tw)

@florbroo        : @baradit Queremos un país sin ustedes payasos.
@hnash46850612   : @patriciapolitz @convencioncl @sebastian_gray @lanetacl @mariomarcelc Ahora seremos un país  desarrollado. 🤣🤣🤣
@soydelever69    : Porque así lo pedimos y queremos un país Justo y Solidario #LaNuevaConstitucionEsNuestra
@Ehitan          : @tere_marinovic Ósea seremos un país fascista?
@vpquinones      : @berfontaine "Queremos un país de iguales"..... ya.
@eljovenmanosde  : @CarolCBown seremos un país bakan😊
@Gingleve        : @Jaime_Bassa NO QUEREMOS UN PAÍS INDIGENISTA. SOMOS TODOS CHILENOS.
#RechazoElPlurimamarracho
@dangubbly       : @christianpviera Claro, un Chile de ultra izquierda. Despierta Chile !!!! No queremos un país de ultra izquierda para la@ultra izquierda.
@RaulMichellod   : @IgnacioAchurra Achurra falso falso la mayoría de los chilenos no queremos un país indigenista vamos a rechazar
@XJefede_latribu : @berfontaine Y remata "Queremos un país de iguales"
@osvaldoeirl     : @gdominguez_ L

In [119]:
def get_similar_user(name, t):

    # Dos usuarios son similares si tienen un gran numero de tweets similares
    user_tweets = clean_tweets[clean_tweets['screen_name'] == name]
    similar_user = {}
    for tweet_id in user_tweets.index:
        # Se obtienen todos los tweets similares a algun tweet del usuario
        tweets_similar = get_similar_tweets(tweet_id, t)
        for tweet in tweets_similar:
            user = clean_tweets.loc[tweet]['screen_name']
            if user != name:
                similar_user.setdefault(user, 0)
                similar_user[user] += 1

    # Se seleccion el usuario con el que mayor frecuencia de tweets similares 
    # haya tenido
    max = 0
    username = ""
    for user in similar_user:
        if similar_user[user] > max:
            max = similar_user[user]
            username = user
    return username


In [120]:
t = 0.1
get_similar_user('ConySchon', t)

'AbrigoCD'

In [121]:
clean_tweets[clean_tweets['screen_name'] == 'ConySchon']

,screen_name,text,preprocess
id,,,
1513648499417567235,ConySchon,Aquí pueden ver el articulado completo :) http...,aquí pueden ver el articulado completo :)
1516574419866591234,ConySchon,Aquí el articulado completo🙌🏼 https://t.co/986...,aquí el articulado completo🙌🏼
1519440649971539969,ConySchon,Aclaro que equivoque en mi votación. Aquí el o...,aclaro que equivoque en mi votación. aquí el o...
1517310579018510341,ConySchon,En la nueva constitución: \n\n2. Hay derecho h...,en la nueva constitución: 2. hay derecho human...
1517310447518683136,ConySchon,En la nueva constitución: \n\n1. Hay derecho a...,en la nueva constitución: 1. hay derecho a la ...
1519443350226620419,ConySchon,Aclaro que me equivoque en mi votación. Aquí e...,aclaro que me equivoque en mi votación. aquí e...


In [122]:
clean_tweets[clean_tweets['screen_name'] == 'AbrigoCD']

,screen_name,text,preprocess
id,,,
1513734937618399234,AbrigoCD,@derechaazul @RojasDcv @CBorgono @HarryJurgens...,en el que se defina. no sé en qué le afecta.
1529914833952710656,AbrigoCD,@EconomistaFlait @mank5_ @fernando_atria Ponte...,ponte serio ql
1529821986909696002,AbrigoCD,@FernaSotoV @adrianovaroli @fernando_atria Y f...,y fueron rechazadas.se presentaron de nuevo ah...
1529820747455004673,AbrigoCD,@FernaSotoV @adrianovaroli @fernando_atria Tie...,tiene link pa leerlo? no lo encuentro por ning...
1510994342009196551,AbrigoCD,@yikinloo @Molinas1321 @RenatoGarinG Ya le pid...,ya le pidieron disculpas públicas
...,...,...,...
1542251684021252098,AbrigoCD,@DonWeasDoctor @ELMISMOKUFO @nmaureir @fernand...,no hace ninguna distinción en cuanto a qué der...
1542223833704300546,AbrigoCD,@incorrecto123 @jorgemiresmunoz @fernando_atri...,en ninguna parte está consagrado el derecho de...
1542225837667368964,AbrigoCD,@incorrecto123 @jorgemiresmunoz @fernando_atri...,exacto. hay gente que no es dueña de la casa e...


In [124]:
t = 0.1
get_similar_user('AbrigoCD', t)

'matiasfortuno'

In [126]:
clean_tweets[clean_tweets['screen_name'] == 'matiasfortuno']

,screen_name,text,preprocess
id,,,
1513291319753515010,matiasfortuno,@patriciapolitz @convencioncl @sebastian_gray ...,"su información, es fake news."
1509702551217782789,matiasfortuno,"@patriciapolitz ""En picado"", querrá decir.","""en picado"", querrá decir."
1514054514910838785,matiasfortuno,@bdelamaza ¿Van a cambiar todas las posturas r...,¿van a cambiar todas las posturas radicales ya...
1511394060451336192,matiasfortuno,"@24HorasTVN @patriciapolitz Muy bien dicho, si...","muy bien dicho, siga así."
1510652496498339846,matiasfortuno,"@patriciapolitz @claudiaheiss @latercera ""Y po...","""y por eso es que nos pasamos x la r las propu..."
...,...,...,...
1541279138635563009,matiasfortuno,@Andres_ArvS @fernando_atria Eso explíqueselos...,eso explíqueselos a sus convencionales. por qu...
1541297922079424512,matiasfortuno,"@Andres_ArvS @fernando_atria En resumen, ¿está...","en resumen, ¿está de acuerdo en que, bajo la n..."
1541516733499965443,matiasfortuno,@Andres_ArvS @fernando_atria Un abrazo!!,un abrazo!!


# Referencias

La solución propuesta toma como referencia:
- [Locality Sensitive Hashing (LSH): The Illustrated Guide](https://www.pinecone.io/learn/locality-sensitive-hashing/)
- [Actividad 6 LocallySensitiveHashing](https://github.com/IIC2440/Syllabus-2023-1/tree/main/Actividades/06%20-%20LocallySensitiveHashing)